<a href="https://colab.research.google.com/github/hochschule-pforzheim/project-st23-team-b23/blob/main/Dummykodierung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In diesem Notebook möchten wir unseren Datensatz für das tranieren der Modelle vorbereiten. Dazu müssen zum Beispiel einige Variablen Dummy kodieren.

In [ ]:
import pandas as pd

df = pd.read_csv('/content/DF_Join_final_bereinigt.csv')


In [ ]:
#Zunächst entfernen wir noch die Spalte imdb_id, die wir nicht ins Modell einfließen lassen

df = df.drop(['imdb_id'], 1)

<ipython-input-2-b7e5dc2987c8>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(['imdb_id'], 1)


#Dummy kodieren Genres und Production Countries

Leonie

In [ ]:
#In der Spalte Genres wollen wir jeweils nur den ersten Eintrag behalten

df['genres'] = df['genres'].str.split(',').str[0]

In [ ]:
#Für eine schönere Darstellung entfernen wir noch die Sonderzeichen

import re

def clean_Sonderzeichen(values):
    cleaned_values = []
    for x in values:
        cleaned_value = re.sub(r'[^a-zA-Z]', '', x)
        if cleaned_value:
            cleaned_values.append(cleaned_value)
    return ''.join(cleaned_values)

# Wir wenden die Funktion für die Spalte genres und production_countries an

df['genres'] = df['genres'].apply(lambda x: clean_Sonderzeichen(x))

df['production_countries'] = df['production_countries'].apply(lambda x: clean_Sonderzeichen(x))


###Zunächst Dummy kodieren wir die Spalte genres

In [ ]:
#Wir haben mehr als zwei Ausprägungen, daher nutzen wir One Hot Encoding

df.genres.value_counts()

drama            15551
comedy           10574
action            4252
thriller          4190
documentation     3044
romance           2944
crime             2404
scifi             2390
animation         1685
horror            1274
fantasy           1238
family             812
war                607
music              537
reality            318
history            215
western            208
sport               75
                     2
Name: genres, dtype: int64

In [ ]:
#Wir haben zwei Spalten, die keinem Genre zugeordnet sind, daher füllen wir diese manuell auf

df[df['genres'] == '']

,name,character,role,title,description,runtime,genres,production_countries,imdb_score,imdb_votes,tmdb_score
13853,David Letterman,Self - Host,ACTOR,My Next Guest Needs No Introduction With David...,TV legend David Letterman teams up with fascin...,50.0,,US,7.8,5581.0,7.6
22977,Carol Burnett,Herself - Host,ACTOR,A Little Help with Carol Burnett,In this unscripted series starring comedy lege...,24.0,,US,6.3,237.0,6.2


In [ ]:
"""Die beiden Filme werden als TV Show eingeordnet, da wir diese Ausprägung aber bisher nicht haben setzen wir beide
Filme auf die nächst passenede Ausprägung documentation"""

#Wir setzen das Genre für den ersten Film auf documentation

rows = df[df['name'] == 'David Letterman']

for index, row in rows.iterrows():
    if row['genres'] == '':
        df.loc[index, 'genres'] = 'documentation'

#Das Genre für den zweiten Film setzen wir auch auf documentation

rows_2 = df[df['name'] == 'Carol Burnett']

for index, row in rows_2.iterrows():
    if row['genres'] == '':
        df.loc[index, 'genres'] = 'documentation'

In [ ]:
#Unser Code war erfolgreich, die 2 Ausprägungen ohne Genre sind weg und bei documentation sind 2 Werte mehr
df.genres.value_counts()

drama            15551
comedy           10574
action            4252
thriller          4190
documentation     3046
romance           2944
crime             2404
scifi             2390
animation         1685
horror            1274
fantasy           1238
family             812
war                607
music              537
reality            318
history            215
western            208
sport               75
Name: genres, dtype: int64

In [ ]:
#Wir wenden nun One Hot Encoding an, löschen die genres Spalte und hängen die neuen an unseren Datensatz an

df = df.drop(columns='genres').join(pd.get_dummies(df.genres, drop_first = True))

###Dummy kodieren Production Countries

In [ ]:
#Wir wiederholen die Schritte für Production Countries

df.production_countries.value_counts()

US        17780
IN         4993
JP         2705
KR         2466
ES         1617
          ...  
USIL          1
EGUS          1
USCLMX        1
BEUS          1
ATUS          1
Name: production_countries, Length: 334, dtype: int64

In [ ]:
#Wir haben sehr viele Kategorien, die nur einmal vorkommen um unser Modell nicht unnötig kompliziert zu machen fassen wir diese zusammen
#Damit es überschaubar bleibt setzen wir den Schwellenwert auf 100
#Die Länder kommen schließlich auch häufig vor, da die Filme mehrfach im Datensatz vorkommen, da jeder Schauspieler zu den Filmen genannt ist

threshold = 100

count = df.production_countries.value_counts()

infrequent_categories = count[count < threshold].index

df.loc[df['production_countries'].isin(infrequent_categories), 'production_countries'] = 'Other'

In [ ]:
df.production_countries.value_counts()

US        17780
Other      5890
IN         4993
JP         2705
KR         2466
ES         1617
GB         1561
FR         1068
PH          998
ID          935
TR          931
CA          870
GBUS        722
BR          700
DE          697
IT          625
USGB        614
MX          575
CN          498
PL          480
CAUS        477
TW          446
AU          422
HK          361
TH          356
AR          309
            290
CO          235
NL          221
NG          204
ZA          198
USCA        198
SE          190
DK          173
EG          157
CAGBUS      148
BEFR        147
USAU        136
CL          131
CNHK        124
DEUS        117
XX          114
NO          113
MY          113
FRBE        113
KRUS        102
Name: production_countries, dtype: int64

In [ ]:
#Wir haben noch 290 Werte die keinem Land zugeordnet sind, diese setzten wir ebenfalls auf Other

df['production_countries'] = df['production_countries'].replace('', 'Other')

In [ ]:
df.production_countries.value_counts()

US        17780
Other      6180
IN         4993
JP         2705
KR         2466
ES         1617
GB         1561
FR         1068
PH          998
ID          935
TR          931
CA          870
GBUS        722
BR          700
DE          697
IT          625
USGB        614
MX          575
CN          498
PL          480
CAUS        477
TW          446
AU          422
HK          361
TH          356
AR          309
CO          235
NL          221
NG          204
ZA          198
USCA        198
SE          190
DK          173
EG          157
CAGBUS      148
BEFR        147
USAU        136
CL          131
CNHK        124
DEUS        117
XX          114
NO          113
MY          113
FRBE        113
KRUS        102
Name: production_countries, dtype: int64

In [ ]:
#Wir wenden nun One Hot Encoding an, löschen die production countries Spalte und hängen die neuen an unseren Datensatz an

df = df.drop(columns='production_countries').join(pd.get_dummies(df.production_countries, drop_first = True))

Weitere Dummy Kodierung: Madeleine:

#Zielvariable berechnen
Leonie

In [ ]:
'''Unsere Zielvariable für die Regression ist der imdb und tmdb Score, diese möchten wir in einem gewichteten Mittelwert
(75% imdb score 25% tmdb score) kodieren'''

weight_imdb = 0.75
weight_tmdb = 0.25

df['target_variable'] = ((df['imdb_score'] * weight_imdb) + (df['tmdb_score'] * weight_tmdb))

#Speichern der Daten

In [ ]:
df.to_csv("Daten_kodiert.csv", index=False)